In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import clear_output, display
from io import StringIO

In [ ]:
URL_BASE_23="https://onlineraceresults.com/race/view_race.php?race_id=76939&re_NO=e.g.%2C+1946&re_FN=e.g.%2C+Joe&re_LN=e.g.%2C+Desch&re_CITY=&re_STATE=&re_DIVISION=&submit_action=select_result&race_id=76939"
URL_2023="https://onlineraceresults.com/race/view_race.php?race_id=76939&relist_record_type=result&lower_bound={}&upper_bound={}&use_previous_sql=1&group_by=default"
URL_BASE_22="https://onlineraceresults.com/race/view_race.php?race_id=74868&submit_action=select_result&order_by=default&group_by=default#results"
URL_2022="https://onlineraceresults.com/race/view_race.php?race_id=74868&relist_record_type=result&lower_bound={}&upper_bound={}&use_previous_sql=1&group_by=default"

In [ ]:
URL = URL_2022
URL_BASE= URL_BASE_22
YEAR = 2022
NAME = "indianapolis_" + str(YEAR)


In [ ]:
page_number = 0    
session = requests.Session()
session.get(URL_BASE)
res = session.get(URL.format(page_number * 1000, (page_number +1)*1000 ))
page = BeautifulSoup(res.text)
#page.find('div',id='race-results-table')

In [ ]:
tbl=page.select("div#race-results-table table")

df=pd.read_html(StringIO(str(tbl)))[0]
df

In [ ]:
df_final = None

session = requests.Session()
session.get(URL_BASE)
page_number = 0

while True:
    clear_output()
    display(f"Retrieving page {page_number}")


    res = session.get(URL.format(page_number * 1000, (page_number +1)*1000 ))
    page = BeautifulSoup(res.text)
    tbl = page.select("div#race-results-table table")
    if not tbl:
        break
    df=pd.read_html(StringIO(str(tbl)))[0]

    if df_final is None:
        df_final = df
    else:
        df_final = pd.concat([df_final, df])

    page_number += 1

df_final.shape

In [ ]:
df_final.reset_index(drop=True, inplace=True)

In [ ]:
df_final = df_final[df_final.OVERALL!="OVERALL"]

In [ ]:
df_final.head()

In [ ]:
df_final['Sex']=df_final.DIVISION.str[:1]

In [ ]:
df_final['Age']=df_final.DIVISION.str[1:]

In [ ]:
df_final['Age'].replace('LITE', 30, inplace=True)

In [ ]:

df_final.to_csv(NAME + '_full.csv')
df_final[['Sex','Age','TIME']].to_csv(NAME + '.csv')

In [ ]:
df_final.DIVISION.value_counts()

In [ ]:
df_final.pivot_table(index='DIVISION', columns='Gender', aggfunc='size', fill_value=0)